In [87]:
import json 
import pathlib
import trimesh

In [88]:
mesh_dir = "/home/tlips/Documents/synthetic-cloth-data/synthetic-cloth-data/data/deformed_meshes/TSHIRT/dsd-tshirts-b"
mesh_dir = pathlib.Path(mesh_dir)
print(mesh_dir)

/home/tlips/Documents/synthetic-cloth-data/synthetic-cloth-data/data/deformed_meshes/TSHIRT/dsd-tshirts-b


In [89]:
meshes = mesh_dir.glob("*.obj")
jsons = mesh_dir.glob("*.json")

meshes = list(meshes)
jsons = list(jsons)

In [90]:
coverages = []
for file in jsons:
    json_dict = json.load(file.open())
    coverage = json_dict["area"] / json_dict["flat_mesh"]["area"]
    coverages.append(coverage)


In [91]:
import numpy as np 
coverages = np.array(coverages)

print(sum(coverages > 0.82))

249


In [92]:

# select those meshes with coverage > 0.82
meshes = [meshes[i] for i in range(len(coverages)) if coverages[i] > 0.82]
print(len(meshes))

249


In [93]:
def read_obj_mesh(obj_path: str):
    """loads a tri-mesh obj file and returns vertices and faces"""
    vertices, faces = [], []
    with open(obj_path, "r") as f:
        lines = f.readlines()
    for line in lines:
        # 3D vertex
        if line.startswith("v "):
            vertices.append([float(n) for n in line.replace("v ", "").split(" ")])
        # Face
        elif line.startswith("f "):
            idx = [n.split("/") for n in line.replace("f ", "").split(" ")]
            face = [int(n[0]) - 1 for n in idx]
            assert len(face) == 3
            faces.append(face)
    return vertices, faces

In [94]:
target_dir = mesh_dir.parent / "processed_meshes"
target_dir.mkdir(exist_ok=True)
import shutil

for mesh in meshes:
    json_path = mesh.with_suffix(".json")
    json_dict = json.load(json_path.open())
    keypoint_vertices = json_dict["keypoint_vertices"]
    print(keypoint_vertices)
    vertices, faces = read_obj_mesh(mesh)
    keypoints = {key: vertices[val] for key,val in keypoint_vertices.items()}

    shutil.copy(mesh, target_dir)
    json.dump(keypoints, (target_dir / f"{mesh.stem}_keypoints.json").open("w"))
    

    

{'shoulder_left': 111, 'neck_left': 116, 'neck_right': 121, 'shoulder_right': 126, 'sleeve_right_top': 131, 'sleeve_right_bottom': 136, 'armpit_right': 141, 'waist_right': 146, 'waist_left': 151, 'armpit_left': 156, 'sleeve_left_bottom': 161, 'sleeve_left_top': 166}
{'shoulder_left': 111, 'neck_left': 116, 'neck_right': 121, 'shoulder_right': 126, 'sleeve_right_top': 131, 'sleeve_right_bottom': 136, 'armpit_right': 202, 'waist_right': 146, 'waist_left': 151, 'armpit_left': 156, 'sleeve_left_bottom': 161, 'sleeve_left_top': 166}
{'shoulder_left': 111, 'neck_left': 116, 'neck_right': 121, 'shoulder_right': 126, 'sleeve_right_top': 131, 'sleeve_right_bottom': 136, 'armpit_right': 141, 'waist_right': 146, 'waist_left': 151, 'armpit_left': 156, 'sleeve_left_bottom': 161, 'sleeve_left_top': 166}
{'shoulder_left': 111, 'neck_left': 116, 'neck_right': 121, 'shoulder_right': 126, 'sleeve_right_top': 131, 'sleeve_right_bottom': 136, 'armpit_right': 141, 'waist_right': 146, 'waist_left': 151, 'ar